In [ ]:
# cell for importing packages for the first time
# import Pkg; Pkg.add("StaticArrays")
# import Pkg; Pkg.add("Revise")


# import Pkg; Pkg.add("@time")
Pkg.add("FastGaussQuadrature")

In [ ]:
ENV["PYTHON"]=""
import Pkg;
Pkg.add("PyCall")

In [ ]:
Pkg.add("PyPlot")

In [ ]:
using Plots

using StaticArrays
import LinearAlgebra
import FastGaussQuadrature

push!(LOAD_PATH, pwd())

module PhysConst
    export CL,Gr,QE,MSOL,MSUN,ARAD,SGMB,RGAS,
            PC,RSUN,RSUN,YR,MSOLYR,GAM53,KPE,PARSEC,KPE,KPD,M_MW
    CL = 2.997925E10;
    Gr = 6.67384E-8; 
    QE=4.80325E-10;
    MSOL = 1.989E33;
    MSUN = 1.989E33;

    ARAD = 7.56464E-15; 
    SGMB = ARAD*CL/4
    
    KBLZ = 1.3807E-16 

    HPL = 6.6261E-27 
    HPLB= 1.0546E-27

    RGAS = 8.31E7; 
    PC = 3.085678E18;
    MP = 1.672661E-24; 
    M_U = 1.660531E-24
    RSOL = 6.95E10
    RSUN = 6.95E10
    YR = 365*24*3600.
    MSOLYR = 1.989e33/YR

    GAM = 5/3.

    KPE = 0.4
    PARSEC=3.0E18
    KPD = 10.

    M_MW=1.

    tst=1

end


#-------------------------------------- STEP 1 -------------------------------------- #
# define necessary data structures

# module SpaceRadGrid

        
# gaussnodes -   Gauss-Legendre nodes
# gaussweights - Gauss-Legendre weights



using StaticArrays
import Base.@kwdef

    const IX = 1
    const IY = 2
    const IZ = 3

    @show const BIGNUMBER = 1.0E50



    @kwdef struct SGridPar
        

        Nactive = 10;
    
        nghost = 1 #N of ghost cells at every boundary.
            
        Nx = Nactive;   # N of active cells
        Ny = Nactive;
        Nz = Nactive;
        
        Nxtot = Nx +  2*nghost;
        Nytot = Ny +  2*nghost;
        Nztot = Nz +  2*nghost;
    
        is = 1 + nghost; 
        ie = Nxtot -nghost;        

        js = 1 + nghost;  
        je = Nytot - nghost;
        
        ks = 1 + nghost;
        ke = Nztot - nghost;
            
        x1s = -1.
        x1e = 1.
    
        x2s = -2.
        x2e = 2.

        x3s = -3.
        x3e = 3.
        
    end

    @kwdef struct RGridPar
    
        Nfreq = 10 ; # ν - N bins 
        NumPhi = 5
        NumTheta = 5
    
        Nang = NumPhi*NumTheta    
        N_fre_ang = Nfreq*Nang  
            
    
    end




# ------ Init Space and Radiation grids --------------------------------

sg= SGridPar()
rad = RGridPar()


# radiaion matter grid is offset from the space grid boundaries
roffst = 2

is_r = sg.is + roffst
ie_r = sg.ie - roffst 

js_r = sg.js + roffst
je_r = sg.je - roffst 

ks_r = sg.ks + roffst
ke_r = sg.ke - roffst 


# μ = zeros(Float64, rad.NumTheta)     
# @show gaussnodes, gaussweights = FastGaussQuadrature.gausslegendre(rad.NumTheta);
# @show μ .= gaussnodes

θ = range(0, pi; length = rad.NumTheta)
ϕ   = range(0, 2pi-pi/rad.NumPhi; length = rad.NumPhi)


# ----------------------------------------------------------------------

#                                Grid

x1 = Array{Float64}(undef, sg.Nxtot)
x2 = Array{Float64}(undef, sg.Nytot)
x3 = Array{Float64}(undef, sg.Nztot)

x1b = Array{Float64}(undef, sg.Nxtot)
x2b = Array{Float64}(undef, sg.Nytot)
x3b = Array{Float64}(undef, sg.Nztot)

# ----------------------------------------------------------------------
#                               Gas   

const Tsc=1.0E4

den = zeros(Float64, sg.Nxtot,sg.Nytot,sg.Nztot) 
Tgas = zeros(Float64, sg.Nxtot,sg.Nytot,sg.Nztot) 
OpaCon = zeros(Float64, sg.Nxtot,sg.Nytot,sg.Nztot)

# ----------------------------------------------------------------------
#                             Radiation
νRange = Array{Float64}(undef, rad.Nfreq) 

IrPos = zeros(Float64, rad.N_fre_ang, sg.Nxtot, sg.Nytot, sg.Nztot); # radiation intensity, forward
IrNeg = zeros(Float64, rad.N_fre_ang, sg.Nxtot, sg.Nytot, sg.Nztot); # radiation intensity, backward

J_avrRadInt = zeros(Float64, sg.Nxtot, sg.Nytot, sg.Nztot); # radiation intensity, backward



function MakeXYZstagGrid(x1, x2, x3, x1b, x2b, x3b, sg)  #staggered grid

    # add check sizes
    
    xtmp = range(sg.x1s, sg.x1e; length = sg.Nxtot - 2sg.nghost)
    
#     @show sg.is, sg.ie sg.Nx sg.nghost size(xtmp) size(x1) size(sg.is:sg.ie) size(2:2)
    
    x1[sg.is:sg.ie] .= xtmp[:] 
    x1[sg.is-1] = 2*x1[sg.is]- x1[sg.is+1]
    x1[sg.ie+1] = 2*x1[sg.ie] -x1[sg.ie-1]

    xtmp = range(sg.x2s, sg.x2e; length = sg.Nytot - 2sg.nghost)    

    x2[sg.js:sg.je] .= xtmp[:]     
    x2[sg.js-1] = 2*x2[sg.js]- x2[sg.js+1]
    x2[sg.je+1] = 2*x2[sg.je] -x2[sg.je-1]

    xtmp = range(sg.x3s, sg.x3e; length = sg.Nztot - 2sg.nghost)    

    x3[sg.ks:sg.ke] .= xtmp[:]     
    x3[sg.ks-1] = 2*x3[sg.ks]- x3[sg.ks+1]
    x3[sg.ke+1] = 2*x3[sg.ke] -x3[sg.ke-1]
    
    
    x1b[1] = x1[1] - 0.5(x1[sg.is]-x1[1])    
    for i in sg.is:(sg.ie+1)
        x1b[i] = x1[i-1] + 0.5(x1[i]-x1[i-1])
    end

    x2b[1] = x2[1] - 0.5(x2[sg.js]-x2[1])    
    for j in sg.js:(sg.je+1)
        x2b[j] = x2[j-1] + 0.5(x2[j]-x2[j-1])
    end

    x3b[1] = x3[1] - 0.5(x3[sg.ks]-x3[1])    
    for k in sg.ks:(sg.ke+1)
        x3b[k] = x3[k-1] + 0.5(x3[k]-x3[k-1])
    end
    
    
    
end

mutable struct LongRay
    numOfElm::Int32    
    id::Int32 #id in dir array
    ijkOfCellCrossed :: Matrix{Int32}        
    xyzPos :: Matrix{Float64}
    orig::Int32 #the origin of the ray, as lin index
end


mutable struct LongRay1  #long ray without a dir

    numOfElm::Int32    
    
#     hed :: Vector{Int32}    
#     dir :: Vector{Float64} 
    
    ijkOfCellCrossed :: Matrix{Int32}        
    xyzPos :: Matrix{Float64}
end


mutable struct RayPair
    
    hed ::Vector{Int32}    
    
    dir::Vector{Float64}    
    
    ray_f:: LongRay1 #forward    
    ray_b:: LongRay1 #backward    
    
end



MakeXYZstagGrid(x1,x2,x3,x1b,x2b,x3b,sg)
@show x3
@show x3b

    
NumElemInRayMax = 2*max(sg.Nxtot,sg.Nytot,sg.Nztot)
    
arrOfLongRay = Array{LongRay, 4}(undef, (rad.Nang, sg.Nxtot, sg.Nytot, sg.Nztot)); 
    
# @time rayPairArr = Array{RayPair, 4}(undef, (rad.Nang, sg.Nxtot, sg.Nytot, sg.Nztot)); 
     


function xyzToijk(xin::Float64, yin::Float64, zin::Float64)
    i = argmin( abs.(x1 .-xin )); 
    j = argmin(abs.(x2.-yin)); 
    k = argmin(abs.(x3.-zin))    
    return(i,j,k)
end

# argmin(νRange,1.0E14)


In [ ]:
# import PhysConst, only:HPL

# SPECTRUM

function DefineSpectrum(wtodo)
    
    if wtodo=="BB"

        
         ν_1 = 0.5*3.5e15
         ν_2 = 10*ν_1
        
         νRange  .= range(ν_1,ν_2, length=100);
        
        J_avrRadInt[i,j,k] 

    else        
        error('undefined in "DefineSpectrum"')    
    end 
    
end







function DefinePhysModel(sg::SGridPar, rad::RGridPar)
    
    Tgas .= 7500.0
    
    den .=  10.0.  # cm^-3 of nH + nH0
    
#     den .= 1.0E18
    
    
    
    
end



function Bν(ν::Float64,T::Float64)
    
  return ( 2.0*PhysConst.HPL* ν^3/(PhysConst.CL^2)/
        (exp(PhysConst.HPL*ν/PhysConst.KBLZ/T)-1.) )
    
end


function testBν()
    
    Trange = range(10.0^4, 10.0^5, length= 3)
    
    T=Tsc
    y =[Bν(nu,T) for nu in νRange]

#     @show -Bν(ν, T)
    
    plot(νRange, y, xaxis=:log, yaxis=:log)

#     for T in Trange
#     #     @show T
#         y =[Bν(nu,T) for nu in νRange]

#         plot!(νRange, y, xaxis=:log, yaxis=:log)


#     end

#     current()

end

DefinePhysModel(sg, rad);

@show Bν(1.0E17, 10.0E5)





In [ ]:
function CrossSecPhotIoniz(ν)
    #photo-ionization cross-section

    Z=1.0    
    ν0 = 3.29e15
    
    if ν < ν0
        return(0.0)
    else
        
        ν1 = Z^2 * ν0     
        
        A0 = 6.3E-18
    
        y = ν/ν1
        
        ϵ = sqrt(y-1.0) 
    
        f1 = 1/(1.0 - exp(-2.0*pi/ϵ))
        
        e1 = exp(4.0 - 4.0*atan(ϵ)/ϵ)
           
        a = A0/Z^2*y^(-4)*e1*f1
    
        return(a)        
    end
    
end


function RecomCoef(ν::Float64, Te::Float64)
    
# α1 = 8*Pi*(ν/PhysConst.CL)^2*(PhysConst.HPL^2/(2π*PhysConst.M))
    
    α1 = 10.0^(-14)         
       
end

function CrosSecRecom(ν::Float64, Te::Float64)
    
# α1 = 8*Pi*(ν/PhysConst.CL)^2*(PhysConst.HPL^2/(2Pi*PhysConst.M))
    
#     n_h - cm^-3 of H-atoms in ground state
    
    Σα = RecombCoef(ν, Te)    
    κ_h = n_h * Σα
    
    return(κ_h)
           
end

function EmissivCoef(ν::Float64, Te::Float64)
   #recombintaion emissivity 
#     Σα
#     n_e * n_io
    
    jν = RecombCoef * /(4π)
    
end



function GetIonFracFromPhotEqu(n_h::Float64,ν::Float64, Te::Float64 )

    Σα = RecombCoef(ν, Te)
    
    IntJ = 
    
    ξ_h = n_h * Σα/IntJ
        
end    
    
    
begin
 ν_1 = 0.5*3.5e15
 ν_2 = 10*ν_1
 nuvec = range(ν_1,ν_2, length=100);        
 y =[CrossSecPhotIoniz(ν) for ν in nuvec];
 y .= y/10^-18
end

plot(nuvec, y)





In [ ]:
pi

In [ ]:
# module RadiationTransfer

x3c = Array{Float64}(undef, 3);
xn = Array{Float64}(undef, 3)
distToNextCell = fill(BIGNUMBER,3) #used in FirstTimeTraceGridOverOneLongRay()
ijk_pos_s = Array{Int32,1}(undef,3)
ijk_pos = Array{Int64}(undef,3)
ijk_max = Array{Int64}(undef,3)
ijk_min = Array{Int64}(undef,3)
ijk_tmp = Array{Int64}(undef,3)

ijk_max .= [ sg.ie+1, sg.je+1, sg.ke+1 ] #max ijk index 
ijk_min .= [ sg.is, sg.js, sg.ks ] #min index
ijk_tmp .= [0, 0, 0]

xbi_zip= [x1b, x2b, x3b]

# display(xbi_zip)

function FirstTimeTraceGridOverOneLongRay(dir::Vector{Float64}, rayOne::LongRay, sg::SGridPar)

    
    dist = 0.0   
    distToNextCell .= BIGNUMBER
    rayOne.numOfElm = 0.0
    
    xn .= x3c
    
#     println("start: ", xn ,"\n")
        
    itermax = 2 * sg.Nx

    for iter in 1:itermax #max length, may be smaller than actual

#      println("*************************")
#      @show xn iter
#      @show ijk_pos

        for (n_it, norm_i ) in enumerate(dir) #QUESTION: maybe iter over pre-calclulated 1/norms

            xc = x3c[n_it] 

             if iter > 1 

                ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, dir[n_it])

                itmp = ijk_tmp[n_it]
             
                xn[n_it] = xbi_zip[n_it][itmp]                                            


             elseif iter==1 #first step from the middle of the cell

                if norm_i>0                
                   
                    itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, dir[n_it])
                    
                elseif norm_i <0               
                    
                    itmp = ijk_tmp[n_it] = ijk_pos[n_it]  #belongs to the same index                                
                else
                    
                    itmp = ijk_pos[n_it]            
                    
                end

                xn[n_it] = xbi_zip[n_it][itmp]

             end
                # @show itmp

             if norm_i != 0                      
                               
                distToNextCell[n_it] = abs((xn[n_it] - xc)/dir[n_it])

             else
                distToNextCell[n_it] = BIGNUMBER
             end


#         println("iter= ",iter," xn=",  xn, " ===> ", "n_it = ",n_it," \n ",
#                 "distToNextCell[",n_it,"]=",distToNextCell[n_it])

            end #for loop over possible directions
                                
        posIndxToUpdate = argmin( distToNextCell )

        dist = distToNextCell[posIndxToUpdate]


        ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one
    

        xn .= x3c .+ dir*dist   
        
#         xn[posIndxToUpdate] = xbi_zip[posIndxToUpdate][ijk_tmp[posIndxToUpdate]]        
#         println(posIndxToUpdate, "   Corrected: ", xn) 
#         @show dir
#         @show posIndxToUpdate ijk_pos ijk_tmp                
#         println("xprev = ", x3c,   " ", "xaft= ", xn, "\n")
        
#         @show posIndxToUpdate dist

        
        x3c .= xn
        
        if ijk_pos[posIndxToUpdate] == ijk_max[posIndxToUpdate] || 
            
            ijk_pos[posIndxToUpdate] == ijk_min[posIndxToUpdate] || 

            ijk_pos[posIndxToUpdate] == 1 ||             
            
            xn[1] <= x1b[sg.is] || xn[1] >= x1b[sg.ie] ||
            xn[2] <= x2b[sg.js] || xn[2] >= x2b[sg.je] ||
            xn[3] <= x3b[sg.ks] || xn[3] >= x3b[sg.ke] ||                        
            iter > 100 
                        
#             @show ijk_pos
#             println("quit..")
            break; 
        end      

        #update rayOne
        rayOne.numOfElm = iter
        rayOne.ijkOfCellCrossed[:,iter] .= ijk_pos
        rayOne.xyzPos[:,iter] .= xn

        if iter>1 && LinearAlgebra.norm(rayOne.xyzPos[:,iter]-rayOne.xyzPos[:,iter-1])<1E-10
            @show iter, rayOne.ijkOfCellCrossed[:,iter], rayOne.ijkOfCellCrossed[:,iter-1]
            @show dir            
            println( "!!!!!!!!!!!!  distance is very small  !!!!!!! \n")
        error( "error! ")            
        end                                
#         println("iter=", iter,"  ", "xn= ", xn)


    end #over ray

end # FirstTimeTraceGridCalcOneLongRay



function InitRayDirections(rad::RGridPar, sg::SGridPar)
            
    norm = zeros(Float64, rad.Nang, 3);
    
    m=1
    for (i,ph_i) in enumerate(ϕ)
        for (j,th_j) in enumerate(θ)
            
            μj = cos.(th_j)
            sinth_j = sqrt(1-μj^2)
            
            norm[m, IX] = sinth_j *sin(ph_i)
            norm[m, IY] = sinth_j *cos(ph_i)                 
            norm[m, IZ] = μj
                        
        m+=1;            
        end
    end

    return(norm)
end

norm = InitRayDirections(rad,sg)

# *****************************************************************
function PlotRay(ray::LongRay, ax1, ax2)

    
#     @show r0 = [x1[ic],x2[jc],x3[kc]]
#     m = 1
#     @show dir = norm[m,:]
#     ray = arrOfLongRay[m,ic,jc,kc]
#     for i in 1:ray.numOfElm
#         r = ray.xyzPos[:,i]
#     end

    @show y1 = ray.xyzPos[ax2,1:ray.numOfElm]
    @show x1 = ray.xyzPos[ax1,1:ray.numOfElm]
    
    p1 = plot(x1, y1,  seriestype = :scatter)
    p2 = plot!(p1, x1, y1)
        
    display(p2)

    
end 

function testFirstTimeTraceGridOverOneLongRay(rad::RGridPar, sg::SGridPar, swch)
    rayid = rand(1:rad.Nang)
    

    @show dir = norm[rayid,:]
    posInRay = 1 #
    
    if swch=="LongRay"
        testRay = LongRay(NumElemInRayMax, rayid,          
            Array{Int32}(undef,3,NumElemInRayMax),
            Array{Float64}(undef,3,NumElemInRayMax), posInRay )       
    else
        error("swch is not supported")
    end

    
    numTests=1
    for n = 1:numTests
        println( " test :", n )
        
        ijk_pos_s[:] = [argmin(abs.(x1.-0.)), argmin(abs.(x2.-0.)), argmin(abs.(x3.-0.))]; 

        ijk_pos_s[:] = [rand( is_r:ie_r),  rand(js_r:je_r),  rand(ks_r:ke_r) ]    
        
        ijk_pos .= ijk_pos_s

        ip=ijk_pos_s[IX]; jp=ijk_pos_s[IY]; kp=ijk_pos_s[IZ]                
     
        x3c .= [ x1[ip], x2[jp], x3[kp] ]
        
        xyz_s = [ x1[ip], x2[jp], x3[kp] ]


        FirstTimeTraceGridOverOneLongRay(dir, testRay, sg)     
        
      
        
        rprev= zeros(Float64,3)
        
#         @show testRay.numOfElm
        
        if testRay.numOfElm > 0
            rprev = xyz_s
            for i=1:testRay.numOfElm
                  
                ds=LinearAlgebra.norm(testRay.xyzPos[:,i] .- rprev)

                rprev .= testRay.xyzPos[:,i]
                
#                 @show i,ds
            end
            
                   
            xyz_e = testRay.xyzPos[:,testRay.numOfElm]                
            dist = LinearAlgebra.norm( xyz_e .- xyz_s)     


            xyz_0e = xyz_s + dir*dist

            Δ = LinearAlgebra.norm( xyz_e[:].-xyz_0e[:])/
                    LinearAlgebra.norm( xyz_0e[:])


            println("|| rend - rs ||/||rs|| = ",  Δ)    
            if Δ <= 1e-5
                println(" Test Passed ! ")    
            else
                println("𝐧 = ",  dir)                
                println(" xyz_e= ",  xyz_e)                
                println(" xyz_s= ",  xyz_s)                
                println(" xyz_0e= ",  xyz_0e)                    
                println(" dist=", dist)      
                error("Test Failed !")
            end

            
        end
            

        
    end
end

function test2(rad, sg)
    rayid = 2
    rayid = rand(1:rad.Nang)
    posInRay = 1 #
    x0=0. 
    y0=0. 
    z0=-2.
    
    @show dir = norm[rayid,:]

    ax1= 1
    ax2 = 2
    ax3 = 3
    
        
    ray = LongRay(NumElemInRayMax, rayid,          
            Array{Int32}(undef,3,NumElemInRayMax), Array{Float64}(undef,3,NumElemInRayMax),posInRay )       

    
    ic,jc,kc=xyzToijk(x0, y0, z0)                    
    ijk_pos .= ijk_pos_s.=[ic,jc,kc]

    
    @show  ip=ijk_pos_s[IX]; jp=ijk_pos_s[IY]; kp=ijk_pos_s[IZ]                
     
    
    
    x3c .= [ x1[ip], x2[jp], x3[kp] ]
        
        
    xyz_s = [ x1[ip], x2[jp], x3[kp] ]

    FirstTimeTraceGridOverOneLongRay(dir, ray, sg)     

    
    @show ray.xyzPos[ax2,:] ray.numOfElm
  
    PlotRay(ray, ax2, ax3)
    
end



# testFirstTimeTraceGridOverOneLongRay(rad, sg, "LongRay")

test2(rad, sg)


In [ ]:


# **************** Trace grid with long rays  ****************** #
    
println("1st time trace grid with long rays, perfom allcoations etc ..")

function TraceGridAllocRayArr(rad::RGridPar, sg::SGridPar, swch)
    
    dirOld =Array{Float64}(undef,3)
    dirf =Array{Float64}(undef,3)
    dirb =Array{Float64}(undef,3)

    x3c_head=Array{Float64}(undef,3)
    ijk_pos_head =Array{Int32}(undef,3)
    
    Nr=NumElemInRayMax
    posInRay =1 
    
    testRay_f = LongRay(Nr,1, Array{Int32}(undef,3,Nr),Array{Float64}(undef,3,Nr),posInRay)       
    testRay_b = LongRay(Nr,1, Array{Int32}(undef,3,Nr),Array{Float64}(undef,3,Nr),posInRay)       

    
    
    if swch=="All"
        is = is_r; ie = ie_r
        js = js_r; je = je_r 
        ks = ks_r; ke = ke_r         
    elseif swch=="OneCell"
        is=ie=ijk_pos_s[IX]; js=je=ijk_pos_s[IY]; ks=ke=ijk_pos_s[IZ]
    else 
        error("Unknown case in TraceGridAllocRayArr")        
    end
         


    for k=ks:ke
    println("k=",k, " of ",ks,":",ke)
        for j=js:je        

            for i=is:ie    
        


                if swch=="All"
                    ijk_pos_s.= [i,j,k]   
                end   
                
                ijk_pos_head .= ijk_pos_s #remember the origin
                
                for m=1:rad.Nang #, ipos=1:sg.Nxtot, jpos=1:sg.Nytot, kpos=1:sg.Nztot

#      @show m " new angle  ****************"
                    dirf .= norm[m,:]
                    dirb .= -norm[m,:]

                    ip=ijk_pos_s[IX]; jp=ijk_pos_s[IY]; kp=ijk_pos_s[IZ]
           
                    ijk_pos .= ijk_pos_s        
                    x3c .= [ x1[ip], x2[jp], x3[kp] ]
                    
                    x3c_head .= x3c #origin 
                    
# @show "forw"
                    FirstTimeTraceGridOverOneLongRay(dirf, testRay_f, sg)        
                    
                    

                   ijk_pos .= ijk_pos_s           
                    x3c .= [ x1[ip], x2[jp], x3[kp] ]
# @show "back"
                    FirstTimeTraceGridOverOneLongRay(dirb, testRay_b, sg)                                            
        
               

#              @show  ijk_pos dirf
                    
                    # now, when the length of the ray is known, allocate arrOfLongRay[m,i,j,k]

                    Ntot = testRay_f.numOfElm + testRay_b.numOfElm +1
                    
                    origCellId = testRay_b.numOfElm + 1 #knowing origCellId one can calc the origin cell
                    
                    arrOfLongRay[m,ip,jp,kp] = LongRay(Ntot, m,            
                        Array{Int32}(undef,3,Ntot), Array{Float64}(undef,3,Ntot), origCellId)

                    arrOfLongRay[m,ip,jp,kp].id = m


                    for iter = 1:testRay_b.numOfElm
                        ib = testRay_b.numOfElm - iter+1                         
                        arrOfLongRay[m,ip,jp,kp].ijkOfCellCrossed[:,iter] = testRay_b.ijkOfCellCrossed[:,ib]                                    
                        arrOfLongRay[m,ip,jp,kp].xyzPos[:,iter] = testRay_b.xyzPos[:,ib]                                    
                   end                       

                   arrOfLongRay[m,ip,jp,kp].ijkOfCellCrossed[:,origCellId] .= ijk_pos_head
                   arrOfLongRay[m,ip,jp,kp].xyzPos[:,origCellId] .= x3c_head
                    
                    for iter = testRay_b.numOfElm + 2 : Ntot

                        i = iter - testRay_b.numOfElm -1 # 1, 2,3, ... (testRay_f.numOfElm + testRay_b.numOfElm)

                        arrOfLongRay[m,ip,jp,kp].ijkOfCellCrossed[:,iter] = testRay_f.ijkOfCellCrossed[:,i]

                        arrOfLongRay[m,ip,jp,kp].xyzPos[:,iter] = testRay_f.xyzPos[:,i]

                    end


              
                end 
            end
        end
    end
   
    
end

# function distAlongRay
#         if testRay.numOfElm > 0
#             rprev = xyz_s
#             for i=1:testRay.numOfElm
#                   ds=LinearAlgebra.norm(testRay.xyzPos[:,i] .- rprev)

#                 rprev .= testRay.xyzPos[:,i]
                
# #                 @show i,ds
#             end
# end

function testTraceGridAllocRayArr(rad, sg)            

    
#     ijk_pos_s[:] = [rand(sg.is:sg.ie),  rand(sg.js:sg.je),  rand(sg.ks:sg.ke) ]    
#     ijk_pos .= ijk_pos_s             
#    @show  ijk_pos_s

    
#      TraceGridAllocRayArr(rad, sg, "OneCell") #"OneCell"        
#      @show ray =  arrOfLongRay[rand(1:rad.Nang), ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]             
     println(" ************************************ ")     
    
      TraceGridAllocRayArr(rad, sg, "All")
    
      @show is = is_r; 
      @show ie = ie_r
      js = js_r; je = je_r 
      ks = ks_r; ke = ke_r  
    
    @show ray =  arrOfLongRay[rand(1:rad.Nang),rand(is_r:ie_r),  rand(js_r:je_r),  rand(ks_r:ke_r) ] 


    println(" ************************************ ")     
    
    
#      PlotRay(ray, ax1, ax2)

end


testTraceGridAllocRayArr(rad, sg)        





# **************************************************************#    

In [ ]:
function test3()
    rayid = rand(1:rad.Nang)

    x0=0. 
    y0=0. 
    z0=0.
      
    ax1= 1
    ax2 = 2
    ax3 = 3
    
    @show dir = norm[rayid,:]
    
    ic,jc,kc = xyzToijk(x0, y0, z0)                    


    ray =  arrOfLongRay[rayid, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]
    
    
    @show ray.xyzPos[3,:]
    
    PlotRay(ray, ax1, ax3)
    
end
test3()

<h1>
<b>
Numerical
</b>
</h1>

$$   
 \begin{equation}    
    p\frac{d^2u}{dt^2} = u - S\mbox{,}
 \end{equation}
$$  


<b> Matrix  </b> eq: 
$$
-A_i u_{i-1} + B_i u_{i} -C_i u_{i+1} = S_i
$$

Coefficients have a slightly different form compared to <b>RH91</b> for a slightly better representation of 
the numerical derivative on a staggered grid. The source term allows for implicit contribution from the $b_m u_m$ from $S$. 
 
 $$
 y_i \left(-b_m
    +\frac{p}{\delta \tau _i}
    \left(\frac{1}{\delta \tau _{i+\frac{1}{2}}}+\frac{1}{\delta \tau
   _{i-\frac{1}{2}}}\right)
   +1\right)
   -\frac{p y_{i-1}}{\delta \tau _{i-\frac{1}{2}} \delta \tau _i}
   -\frac{p y_{i+1}}{\delta \tau _i \delta \tau _{i+\frac{1}{2}}}-S_i-S_\beta=0
 $$

Boundary conditions:
    
 $$
 \text{at}\quad t = \tau_1\simeq 0: \quad \alpha \, u'(t) + \beta \, u(t) = \gamma \mbox{,} 
 $$
 
 $$
 v(\tau=0)=u(\tau=0)\mbox{,}
 $$
    
 $$
 \text{at}\quad t = \tau_N: \quad \alpha_1 \, u'(t) + \beta_1 \, u(t) = \gamma_1 \mbox{.} 
 $$


 $$
    i=1:\qquad y_1 \left(-\frac{\alpha}{\delta \tau _{\frac{3}{2}}}+\beta\right)+\frac{\alpha}{\delta \tau _{\frac{3}{2}}} y_2=\gamma
 $$
 
 $$
 i=N:\qquad
  -\frac{\alpha_1 }{\delta \tau _{N-\frac{1}{2}} }y_{N-1} +y_N \left(\frac{\alpha_1}{\delta \tau _{N-\frac{1}{2}} }+\beta_1\right)=\gamma_1
$$
 
<h>
    <b> Source function: </b>   
</h>
$$ 
S_{i}=\alpha\,\Sigma_{n}^{N_{\nu}}a_{n}\phi_{in}\,\Sigma_{m=1}^{N_{m}}b_{m}u_{imn}
$$ 
    

<h1> Coeffs and Source Function </h1>

$$   
 \begin{equation}    
    p\frac{d^2u}{dt^2} = u - S\mbox{,}
 \end{equation}
$$  

$$
p=\frac{1}{(\chi_\nu + \sigma_\nu)^2}
$$

In [ ]:
function UpdateOpacGrid(sg::SGridPar)
     
    for k=1:sg.Nztot
        for j=1:sg.Nytot      
            for i=1:sg.Nxtot
                
            OpaCon[i,j,k]=2. 

            end
        end
    end  
            
end

function SrcFun(ν, i::Int32,j::Int32,k::Int32)

    T =  Tgas[i,j,k]
    
    Sf::Float64 = Bν(ν, T)
    
    return(Sf)
    
    
end

function ProgRayBC_1(ν, i::Int32,j::Int32,k::Int32)
    
        
    chi = OpaCon[i,j,k]     

    ν = 1.0E16    
    T=1.0E5


    alph = 1.0 
    

    bet = -chi            

    gam = 0.0 #Bν(ν, T)
        
    return(alph, bet, gam)
    
end
function ProgRayBC_N(ν, i::Int32,j::Int32,k::Int32)
    
    chi = OpaCon[i,j,k] 
    
    alphN = 1.    
    
    betN = chi

    gamN = 0.
    
    return(alphN,betN,gamN)
    
end

function ProgRay_pCoef(ν, i::Int32,j::Int32,k::Int32)
    chi = OpaCon[i,j,k]    
    p = 1/chi^2
 return(p)
end    

function ProgonRybHumAlongTheRay(ν, ray::LongRay, sg::SGridPar, rad::RGridPar)

    Nmat = ray.numOfElm
    
    u = zeros(Float64, Nmat)
    v = zeros(Float64, Nmat)
    
    
    A = zeros(Float64, Nmat)
    B = zeros(Float64, Nmat)    
    C = zeros(Float64, Nmat)
    D = zeros(Float64, Nmat)
    Z = zeros(Float64, Nmat)
    S = zeros(Float64, Nmat) #Rhs


    Sbet =0.
    bm = 0
    

    ic,jc,kc = ray.ijkOfCellCrossed[1,1],ray.ijkOfCellCrossed[2,1],ray.ijkOfCellCrossed[3,1]

    alph,bet,gam = ProgRayBC_1(ν, ic,jc,kc)
    
                       
    dx_i1 = LinearAlgebra.norm(ray.xyzPos[:,2].-ray.xyzPos[:,1])     
    dx_i=dx_i1 
    dx_ip05 = dx_i

    p = ProgRay_pCoef(ν, ic,jc,kc)
            
    A[1] = 0.
    
    B[1] = bet - alph/dx_ip05  
    
    C[1] = -alph/dx_ip05    
    
    S[1] = gam
    
    D[1] = C[1]/B[1]
    Z[1] = S[1]/B[1]
    
    C[Nmat]=0.
    
    for i = 2 : Nmat-1 #forward            
        ic,jc,kc = ray.ijkOfCellCrossed[1,i],ray.ijkOfCellCrossed[2,i],ray.ijkOfCellCrossed[3,i]
        
        dx_i = LinearAlgebra.norm(ray.xyzPos[:,i+1].-ray.xyzPos[:,i])

        if dx_i==0
            @show "warn: " Nmat, i, ic,jc,kc, ray.xyzPos[:,i+1],ray.xyzPos[:,i]
            error("dx=0 in ProgonRybHumAlongTheRay")
        end
            
        dx_im05 = dx_i         
        dx_ip05 = dx_i  

        p = ProgRay_pCoef(ν, ic,jc,kc)
        
        Sd = SrcFun(ν, ic,jc,kc)
        
        S[i] =Sd +Sbet
        
        A[i] = p /(dx_i*dx_im05)         
        B[i] = 1. - bm + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i        
        C[i] = p/(dx_i*dx_ip05)  
                        
        k1 = 1/(B[i] - A[i]D[i-1])        
        D[i] = k1*C[i]        
        Z[i] = k1*( S[i]  + A[i]*Z[i-1])                     
    end 

    ic,jc,kc = ray.ijkOfCellCrossed[1,Nmat],ray.ijkOfCellCrossed[2,Nmat],ray.ijkOfCellCrossed[3,Nmat]    
    
    alphN,betN,gamN = ProgRayBC_N(ν, ic,jc,kc)

    dx_im05 = dx_i         
    dx_ip05 = dx_i 
    S[Nmat] = gamN
    
    C[Nmat]=0.
    D[Nmat]=0.

    A[Nmat] = (alphN/dx_im05)
    B[Nmat] = betN + alphN/dx_im05
    
    k1 = 1/(B[Nmat] - A[Nmat]*D[Nmat-1])
    
    Z[Nmat] = k1*( S[Nmat]  + A[Nmat]*Z[Nmat-1])                     

    u[Nmat] = Z[Nmat]
    v[Nmat] = u[Nmat]
    
    for i = Nmat-1:-1:1  #backsweep

        u[i] = D[i]*u[i+1]+Z[i]
        
        dx_i = LinearAlgebra.norm(ray.xyzPos[:,i+1].-ray.xyzPos[:,i])
        
         duds = (u[i+1]-u[i])/dx_i   
                
        ic,jc,kc = ray.ijkOfCellCrossed[1,i],ray.ijkOfCellCrossed[2,i],ray.ijkOfCellCrossed[3,i]
                
        chi = OpaCon[ic,jc,kc] 

        v[i] = -1.0/chi*duds
        
    end #for  
    #   calculate v at the origin     

#     i1 = ray.orig    
#     ic,jc,kc = ray.ijkOfCellCrossed[1,i1],ray.ijkOfCellCrossed[2,i1],ray.ijkOfCellCrossed[3,i1]
#     dx_i = LinearAlgebra.norm(ray.xyzPos[:,i1+1].-ray.xyzPos[:,i1])    
#     duds = (u[i1+1]-u[i1])/dx_i        
#     chi = OpaCon[ic,jc,kc]    
    
  
    
    return(u,v)
end #RTAlongTheRay

function ProgonRybHumAlongTheRayAppndxA(ν, ray::LongRay, sg::SGridPar, rad::RGridPar)
    
    Nmat = ray.numOfElm

    u = zeros(Float64, Nmat)
        
    D = zeros(Float64, Nmat)
    H = zeros(Float64, Nmat)
    F = zeros(Float64, Nmat)
    Z = zeros(Float64, Nmat)    


    
    Sbet =0.
    bm = 0.
    
    ic,jc,kc = ray.ijkOfCellCrossed[1,1],ray.ijkOfCellCrossed[2,1],ray.ijkOfCellCrossed[3,1]
    
    alph,bet,gam = ProgRayBC_1(ν, ic,jc,kc)
    
    dx_i1 = LinearAlgebra.norm(ray.xyzPos[:,2].-ray.xyzPos[:,1])     
    dx_i=dx_i1 
    dx_ip05 = dx_i
        
    p = ProgRay_pCoef(ν, ic,jc,kc)

    
    B1 = -bet + alph/dx_ip05
    C1 = alph/dx_ip05
    S1 = gam

    H[1] = B1-C1    
    F[1] = H[1]/C1
    Z[1] = S1/B1    
    
    
    for i = 2 : Nmat-1 #forward                        
        ic,jc,kc = ray.ijkOfCellCrossed[1,i],ray.ijkOfCellCrossed[2,i],ray.ijkOfCellCrossed[3,i]
        
        
        dx_i = LinearAlgebra.norm(ray.xyzPos[:,i+1].-ray.xyzPos[:,i])     
     
        if dx_i==0
            @show "warn: " Nmat, i, ic,jc,kc, ray.xyzPos[:,i+1],ray.xyzPos[:,i]    
            error("dx=0 in ProgonRybHumAlongTheRay")        
        end
       
        dx_im05 = dx_i         
        dx_ip05 = dx_i  

        p=ProgRay_pCoef(ν, ic,jc,kc)
        
        Sd = SrcFun(ν, ic,jc,kc)
        
        Si = Sd+Sbet
        
        Ai = p /(dx_i*dx_im05)         
        Bi = 1 - bm + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i        
        Ci = p/(dx_i*dx_ip05)  
        H[i] = -Ai + Bi - Ci                
        F[i]=(H[i]+(Ai*F[i-1])/(1+F[i-1]))/Ci        
        Z[i]=(Si+Ai*Z[i-1])/(1+F[i])/Ci      
        
    end 


    ic,jc,kc = ray.ijkOfCellCrossed[1,Nmat],ray.ijkOfCellCrossed[2,Nmat],ray.ijkOfCellCrossed[3,Nmat]    
u
    alphN,betN,gamN = ProgRayBC_N(ν, ic,jc,kc)

    dx_im05 = dx_i         
    dx_ip05 = dx_i 
    SN = gamN
    
    CN=0.
#     @show 
    AN = (alphN/dx_im05)
#     @show 
    BN = betN + alphN/dx_im05
    
#     @show 
    Dnm1 = 1/(1.0+F[Nmat-1])    
    
    Z[Nmat] = (SN+AN*Z[Nmat-1])/( BN - AN*Dnm1)                         
    u[Nmat] = Z[Nmat]

    for i = Nmat-1:-1:1  #backsweep

        u[i] = u[i+1]/(1+F[i]) + Z[i]
        
    end #for                    
#     println("... ProgonRybHumAlongTheRayAppndxA done")
   
    i1=1
    ic,jc,kc = ray.ijkOfCellCrossed[1,i1],ray.ijkOfCellCrossed[2,i1],ray.ijkOfCellCrossed[3,i1]

    dx_i = LinearAlgebra.norm(ray.xyzPos[:,i1+1].-ray.xyzPos[:,i1])
    
    duds = (u[2]-u[1])/dx_i    
    
    chi = OpaCon[ic,jc,kc]    
    
    v = -1/chi*duds  
    
    return(u,v)
    
end #RTAlongTheRay


# tests
function TestRybHumAlongTheRay(rad)
    # pick a ray
    
    ijk_pos_s[:] = [rand(sg.is:sg.ie),  rand(sg.js:sg.je),  rand(sg.ks:sg.ke) ]    
    ijk_pos_s[:] = [rand( is_r:ie_r),  rand(js_r:je_r),  rand(ks_r:ke_r) ]    
    
#     ijk_pos_s[:] = [sg.is,  sg.je,  rand(sg.ks:sg.ke) ]    
    
    ijk_pos_s[:] = [is_r,  je_r,  rand(ks_r:ke_r) ]    
    
    NumTests=1
    
    Δ=0.
    

    for k=1:NumTests
        
        ijk_pos .= ijk_pos_s
        m = rand(1:rad.Nang)
        
#         m=1
         @show dir = norm[m,:]
        
        #need to first run=: TraceGridAllocRayArr(rad, sg)            
        ray =  arrOfLongRay[m, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]
        @show ray.numOfElm
        
        
#         if ray.numOfElm > 1        

        s = zeros(Float64,ray.numOfElm) #dist along the ray
        
        s[1]=0.

            for i = 2:ray.numOfElm
               s[i] =  s[i-1] + LinearAlgebra.norm(ray.xyzPos[:,i]-ray.xyzPos[:,i-1])      
            end
        
#     end
        @show s
        
        UpdateOpacGrid(sg)

        @show νi = νRange[6]
        
        u1,v1 = ProgonRybHumAlongTheRay(νi, ray, sg, rad)    
        
        u,v = ProgonRybHumAlongTheRayAppndxA(νi, ray, sg, rad);    
        
#         p1 = plot(s,u1)

        p2 = plot!( plot(s,u1,label = "u"), s,v1, title = "Feautrier sol", label = "v")
        
        display(p2)    
        

      @show length(u1)

        Δ += LinearAlgebra.norm((u-u1)./u1) + LinearAlgebra.norm((v-v1)./v1)
    println( "relative difference between ProgonRybHumAlongTheRayAppndxA 
        and ProgonRybHumAlongTheRay:\n Δu/u=", Δ )    

    if Δ <= 1e-2
        println(" Test Passed ! ")    
    else
        @show ijk_pos_s
        @show u 
        @show u1
        @show v1
        @show ray.ijkOfCellCrossed 
        @show ray.xyzPos
            
        plot(u)
            
#   error( " Test Failed !")
    end
        
    end



end







TestRybHumAlongTheRay(rad)



In [ ]:
function AvrIntensAllGrid(rad::RGridPar)

    y=0.
    
    is = is_r; ie = ie_r    
    js = js_r; je = je_r 
    ks = ks_r; ke = ke_r         
  
    
    for k=ks:ke
        for j=js:je        
            for i=is:ie   
               
                y=0.
                
                for m=1:rad.Nang                               
                    y += (IrPos[m,i,j,k] + IrNeg[m,i,j,k])
                end
                y = y/rad.Nang
                
            end
        end
    end
    
     
    println( "AvrIntensAllGrid done .. ")
    
    return(y)
    
end

function RTSolveAllGrid(sg::SGridPar, rad::RGridPar)
       
   @show  is = is_r; ie = ie_r    
    js = js_r; je = je_r 
    ks = ks_r; ke = ke_r         

    UpdateOpacGrid(sg)
    
    
    
    for k=ks:ke
        for j=js:je        
            for i=is:ie   
                for m=1:rad.Nang

                    νi = νRange[1]
                    
  
                    ray =  arrOfLongRay[m,i,j,k]
                    
#                   @show ray.ijkOfCellCrossed[:,ray.orig] ray.ijkOfCellCrossed[:,ray.orig+1]
                    
                    u0, v1 = ProgonRybHumAlongTheRay(νi, ray, sg, rad);                    
                    
                    i1 = ray.orig
                    
                    u1 = u0[i1]

                    #display(plot(s,u1))

                    
                    IrPos[m,i,j,k] = u1 + v1
                    IrNeg[m,i,j,k] = u1 - v1   
                    
#                     @show IrPos[m,i,j,k] IrNeg[m,i,j,k] 
                    
                end
    
    
            end
        end
    end
   
    println("RTSolveAllGrid done ..")
end

# 1
RTSolveAllGrid(sg,rad)

# 2
AvrIntensAllGrid(rad)


In [20]:

sizeof(Int8)
    

1

In [ ]:
using Plots
import GR

function PlotRad(x::Float64, y::Float64, z::Float64)
    @show ic,jc,kc = xyzToijk(x,y,z)
    
    y = IrPos[:,ic,jc,kc]
    
    @show size(y), size(μ) rad.Nang
    
    plot(1:rad.Nang, y)
    
end


PlotRad(0.,0.,0.)

In [ ]:
#  using PyPlot 
# using Plots
# gr() 


using Plots
import GR
gr()

function RTSolveTwoWayRay(RayIndex::Int64)        
   
   
    
    # pick a ray
    rayf =  rayPairArr[RayIndex, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]].ray_f
    uf  = ProgonRybHumAlongTheRayAppndxA(rayf, sg, rad); #forward
    
    
#     @show RayIndex, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]
    # @show rayPairArr[RayIndex, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]].dir

  
    #     change direction                
     rayb =  rayPairArr[RayIndex, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]].ray_b
     ub  = ProgonRybHumAlongTheRayAppndxA(rayb, sg, rad); #backward    
      
    utot = zeros(Float64,length(ub)+length(uf))    
#     utot[1:length(ub)] .= ub
#     plot(utot);    
    
    
    @show ub[1] uf[1]

    u = 0.5(uf[1] + ub[1])
    
        

    
end 


    
ijk_pos_s[:] = [rand(sg.is:sg.ie),  rand(sg.js:sg.je),  rand(sg.ks:sg.ke) ]
    
ijk_pos_s[:] = [argmin(abs.(x1.-0.)), argmin(abs.(x2.-0.)), argmin(abs.(x3.-0.))];   



TraceGridBothWaysAllocateArr(rad, sg)

for m=1:rad.N_fre_ang 
    den .= 1.
    opa .= 1.
    RTSolveTwoWayRay(m)
end


<h1>
    <b>
   Schwarzschild-Schuster 2BV problem:
    </b>
    
</h1>

<!-- <font size="4"> -->



$$   
 \begin{equation}    
    p\frac{d^2u}{dt^2} = u - S\mbox{,}
 \end{equation}
$$  
where $t=\tau$, $p=\mu^2 = const$, $S=const$
    
Boundary conditions:
    
 $$
 \text{at}\quad t = \tau_1\simeq 0: \quad \alpha \, u'(t) + \beta \, u(t) = \gamma \mbox{,} 
 $$
    
 $$
 \text{at}\quad t = \tau_N: \quad \alpha_1 \, u'(t) + \beta_1 \, u(t) = \gamma_1 \mbox{.} 
 $$
Source function in SS-approximation:
\begin{equation}
    S(t) = \frac{1}{2}\left( I^{+}(t) + I^{-}(t) \right)\equiv u(t)
\end{equation}


<!-- </font> -->


In [ ]:
function DensMatrixSolvAlongTheRay(xGrid, rayOne::LongRay1, sg::SGridPar, rad::RGridPar)
    
    
    Nn = rad.Nfreq; # ν - N bins 
    Nd = rayOne.numOfElm; # x - bins 
    Nmat = Nd*Nn
    
    Mat = zeros(Float32, Nmat, Nmat)
    Rhs = zeros(Float32, Nmat) #right side    
    
    alph = 1.
    bet = -1.
    gam = 0

    alph1 = 1.
    
    bet1 = 1.
    gam1 = 1.

    Sd = 2.
    Sbet =1.
    bm = 1.
    
    p=1.


    for i = 1:Nmat
        for i = 1:Nmat

            iSubMatRaw = div(i + Nn-1, Nn)         

            if iSubMatRaw == 1 
                Rhs[i] = -gam                   
            elseif iSubMatRaw == Nd 
                Rhs[i] = gam1
            else
                Rhs[i] = Sd+Sbet
            end


            for j = 1:Nmat


            jSubMatCol = div(j + Nn-1, Nn)

            iloc = i - Nn * (iSubMatRaw-1)        
            jloc = j - Nn * (jSubMatCol-1)

            if iSubMatRaw == 1 #left BC    


                dx_i = xGrid[2]-xGrid[1]
                dx_im05 = dx_i 
                dx_ip05 = dx_i        
                fPar = [dx_im05, dx_i, dx_ip05, 1]

                if jSubMatCol == 1
                    B1 = -bet + alph/dx_ip05

                    Mat[i,j] = iloc==jloc ? B1 : 0 #only diag    

                elseif jSubMatCol == 2

                    C1 = -alph/dx_ip05

                    Mat[i,j] = iloc==jloc ? C1 : 0

                end                        

            elseif iSubMatRaw > 1 && iSubMatRaw < Nd
    #           the main body of the matrix

                dx_i = xGrid[iSubMatRaw+1]-xGrid[iSubMatRaw]
                dx_im05 = dx_i 
                dx_ip05 = dx_i

                    if iSubMatRaw == jSubMatCol  && iloc==jloc
#                         B = 1 + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i
                        
                        B = 1 - bm + ((1/dx_im05 + 1/dx_ip05)*p)/dx_i
                        
                        Mat[i,j] = B
                    elseif iSubMatRaw == jSubMatCol+1 && iloc==jloc                     
                        A =  -p /(dx_i*dx_im05)                     
                        Mat[i,j] = A                    
                    elseif iSubMatRaw == jSubMatCol-1                    
                        C=-(p/(dx_i*dx_ip05))                                        
                        Mat[i,j] = C
                    else                                        
                        Mat[i,j] = 0                                                        
                    end


            elseif iSubMatRaw == Nd #right BC                
                dx_i = xGrid[Nd]-xGrid[Nd-1]
                dx_im05 = dx_i 
                dx_ip05 = dx_i

                if jSubMatCol == Nd-1

                    AN = -(alph1/dx_im05)

                    Mat[i,j] = iloc==jloc ? AN : 0

                elseif jSubMatCol == Nd

                    BN = bet1 + alph1/dx_im05

                    Mat[i,j] = iloc==jloc ? BN : 0

                end

            end

            end


        end
    end

# end

    display(Mat)
    # display(Rhs)
    # @show Rhs
    Mat_inv = inv(Mat)
    # Mat_inv*Mat

    ytestNumeric1 = Mat_inv*Rhs

    
    
    end  #RTFotrieAlongTheRay

    
function SchwarzSchustRT()

        ray = arrOfLongRay[1, ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]
    
        @show  ray.dir
                    
        tauGrid = range(0., 1; length = ray.numOfElm)    
        #typeof(tauGrid)
    
    
        DensMatrixSolvAlongTheRay(tauGrid, ray, sg, rad)
    

    
end



SchwarzSchustRT()


In [ ]:

# module RadTransfer
    

# ijk_pos = Array{Int64}(undef,3)
# ijk_max = [ sg.ie+1, sg.je+1, sg.ke+1 ] #max ijk index 
# ijk_min = [ sg.is, sg.js, sg.ks ] #min index

# distToNextCell = fill(BIGNUMBER,3) #used in FirstTimeTraceGridOverOneLongRay()


# norm[1, IX, :, :, :] .=  1
# norm[1, IY, :, :, :] .= -1
# norm[1, IZ, :, :, :] .= -1


# iOf_0 = argmin(abs.(x1)); jOf_0 = argmin(abs.(x2)); kOf_0 = argmin(abs.(x3))
# # ijk_pos .= [sg.is, sg.js, sg.ks]
# # ijk_pos .= [iOf_0, div(sg.je,2), div(sg.ke,2)]
# # ijk_pos .= [sg.ie, sg.je, sg.ke]

# ijk_pos .= [iOf_0,  jOf_0,  kOf_0]
# ijk_tmp = [0, 0, 0]



# rayOne.dir .= [norm[1, IX, 1, 1, 1], norm[1, IY, 1, 1, 1] , norm[1, IZ, 1, 1, 1]]


# rayOne.dir=LinearAlgebra.normalize(rayOne.dir)

# @show rayOne.dir
# @show ijk_pos

# xbi_zip = [x1b, x2b, x3b]



# x3c = [ x1[ijk_pos[IX]], x2[ijk_pos[IY]], x3[ijk_pos[IZ]] ]

# # @show x3c
# # @show x1
# # @show x1b

# # @show x2
# # @show x2b

# # @show x3
# # @show x3b

# # @show xbi_zip[2]


# FirstTimeTraceGridOverOneLongRay(rayOne, sg)







# end #module RadTransfer

In [ ]:
#-------------------------------------- STEP 2 -------------------------------------- #



# @show size(norm);



In [ ]:
# logger=Logging.SimpleLogger(stderr,Logging.Debug)

function TestOneRay(ray, sg)
    xyz_s=Vector{Float64}(undef,3)    
    xyz_e=Vector{Float64}(undef,3)    
    xyz_0e=Vector{Float64}(undef,3)
    
    xyz_s .= ray.xyzPos[:,1]
    
    xyz_e = ray.xyzPos[:,ray.numOfElm]

    dist =LinearAlgebra.norm( xyz_e[:].-xyz_s[:]) 
    
    xyz_0e = xyz_s + ray.dir*dist
    
    
    @show xyz_e, xyz_0e, dist
    
    println("|| rend - rs || = ",  LinearAlgebra.norm( xyz_e[:].-xyz_0e[:]) )
    
end


TestOneRay(rayOne, sg)



In [ ]:
module debugByPlot

using Plots
# x= LinRange(0,30,10)

# import 

x = [0.5, 1.]
y = [0., 1.]
z = [0., 0.1];


plt=plot3d(x,y,z)



end


In [ ]:
ray = arrOfLongRay[rand(1:rad.Nang), ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]]


function PlotRay(ray, sg)
    
    x = ray.xyzPos[1,1:ray.numOfElm]
    y = ray.xyzPos[2,1:ray.numOfElm]   
    z = ray.xyzPos[3,1:ray.numOfElm]      
    
    @show ray.numOfElm,x,y,z

    plot3d(x,y,z,
        xlabel = "x", ylabel = "y", zlabel = "z", 
        xlim= (sg.x1s,sg.x1e),ylim= (sg.x2s,sg.x2e),zlim= (sg.x3s,sg.x3e))    
end



# plt=PlotRay(rayOne, sg); display(plt)    
# display(plt)    
# inline()
# end #module RadiationTransfer


# typeof(x1)

    
i = argmin(abs.(x1.-0.)); 


# GetIndxGiveCoords1(0., 0., 0., x1, x2, x3)

    
@show ijk_pos_s ray.dir

plt=plot3d(ray.xyzPos[1,1:ray.numOfElm],
        ray.xyzPos[2,1:ray.numOfElm],
        ray.xyzPos[3,1:ray.numOfElm],
        xlabel = "x", ylabel = "y", zlabel = "z", 
        xlim= (sg.x1s,sg.x1e),ylim= (sg.x2s,sg.x2e),zlim= (sg.x3s,sg.x3e))

 
display(plt)
# function xyzToijk(xin::Float64, yin::Float64, zin::Float64, x1::Array{Float64,1}, x2::Array{Float64,1},x3::Array{Float64,1})
#     i = argmin( abs.(x1 .-xin )); 
#     j = argmin(abs.(x2.-yin)); 
#     k = argmin(abs.(x3.-zin))    
#     return(i,j,k)
# end

# typeof(x1)

    
# i = argmin(abs.(x1.-0.)); 


# GetIndxGiveCoords1(0., 0., 0., x1, x2, x3)



# function mtest()
# L = M = N = Int(5e2);

# vec_vec_vec = Array{Array{Vector}}(L);

# for i = 1:L vec_vec_vec[i] = [zeros(N) for j = 1:M] end;

# vec_mat = Array{Matrix}(L);

# fill!(vec_mat, rand(M, N));

# arr = rand(L, M, N);

# @time  for i = 1:L for j = 1:M for k = 1:N vec_vec_vec[i][j][k] += 1; end; end; end; 

# @time  for i = 1:L for j = 1:M for k = 1:N vec_mat[i][j,k] += 1; end; end; end; 

# @time  for k = 1:N for j = 1:M for i = 1:L arr[i,j,k] += 1; end; end; end; 

# end
# mtest()



# mutable struct SegmentData{
#  /*! cell data along the trajectory */
#   /*!  current index of cell along the ray, phi index is the same */
#   /*! if the source is symmetrical */
#   int i,k;    
#   float dl; //length element    
# }SegmentData;

# RayCellInd1 = Array{Int32}(undef,1)
# RayCellInd2 = Array{Int32}(undef,1)
# RayCellInd3 = Array{Int32}(undef,1)
# push!(RayCellInd1, 1)
# RayCellInd1 .=0
# RayCellInd2 .=0
# RayCellInd3 .=0


# create array of ray directions, mu, for every ijk cell (nx,ny,nz) components
# for every ray; 
# nang - number of rays per cell 



# RayData = Array of ( (iCross, jCross, kCross), NcrossCells,(istart, jstart, kstart)=3)
# NcrossCells=3
# @time RayData = zeros(3,  zeros(3),  3)





In [ ]:
function FirstTimeTraceGridOverOneLongRay1(dir::Vector{Float64}, rayOne::LongRay1, sg::SGridPar)


#     @show dir
    
    dist = 0.0   
    distToNextCell .= BIGNUMBER

    xn .= x3c

    itermax = 2 * sg.Nx

    for iter in 1:itermax #max length, may be smaller than actual

    @show iter


        for (n_it, norm_i ) in enumerate(dir) #QUESTION: maybe iter over pre-calclulated 1/norms


          xc = x3c[n_it] 

             if iter > 1 

                itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, dir[n_it])

                @debug  " debug: (itmp, iter)= " , itmp, iter

                xn[n_it] = xbi_zip[n_it][itmp]                                            


             elseif iter==1 #first step from the middle of the cell

                if norm_i>0                
                   itmp = ijk_tmp[n_it] = ijk_pos[n_it] + copysign(1, dir[n_it])                     
                elseif norm_i <0               
                    itmp = ijk_tmp[n_it] = ijk_pos[n_it]  #belongs to the same index                                
                else                
                    itmp = ijk_pos[n_it]            
                end

                xn[n_it] = xbi_zip[n_it][itmp]

             end


             if norm_i != 0                      
                distToNextCell[n_it] = abs((xn[n_it] - xc)/dir[n_it])
             else
                distToNextCell[n_it]=BIGNUMBER
             end


        #println("iter= ",iter," xn=", xn, " ===> ", "distToNextCell[",n_it,"]=",distToNextCell[n_it])



        end #for loop over norm_i     

            posIndxToUpdate = argmin(distToNextCell)
            dist = distToNextCell[posIndxToUpdate]


        ijk_pos[posIndxToUpdate] = ijk_tmp[posIndxToUpdate] #update only relevant one
    

        #println("ijk_pos[posIndxToUpdate = ", posIndxToUpdate, "]",  "  =  ",   ijk_pos  )


        xn .= x3c + dir*dist
        x3c .= xn

        #update rayOne
        rayOne.numOfElm = iter
        rayOne.ijkOfCellCrossed[:,iter] .= ijk_pos
        rayOne.xyzPos[:,iter] .= xn

#         println("iter=", iter,"  ", "xn= ", xn)

        if ijk_pos[posIndxToUpdate] == ijk_max[posIndxToUpdate] || ijk_pos[posIndxToUpdate] == ijk_min[posIndxToUpdate] || ijk_pos[posIndxToUpdate] == 1 || iter > 100 
            break; 
        end      

    end #over ray

end # FirstTimeTraceGridCalcOneLongRay

function TraceGridBothWaysAllocateArrPair(rad::RGridPar, sg::SGridPar)
    
    dirOld =Array{Float64}(undef,3)
    dirf =Array{Float64}(undef,3)
    dirb =Array{Float64}(undef,3)
    
    testRay_f = LongRay1(NumElemInRayMax, Array{Int32}(undef,3,NumElemInRayMax),  Array{Float64}(undef,3,NumElemInRayMax) )       
    testRay_b = LongRay1(NumElemInRayMax, Array{Int32}(undef,3,NumElemInRayMax),  Array{Float64}(undef,3,NumElemInRayMax) )       
#     @show ijk_pos_s            

    for m=1:rad.N_fre_ang #, ipos=1:sg.Nxtot, jpos=1:sg.Nytot, kpos=1:sg.Nztot

        
        dirf .= norm[m,:]
        dirb .= -norm[m,:]
                        
        ip=ijk_pos_s[IX]; jp=ijk_pos_s[IY]; kp=ijk_pos_s[IZ]
        ijk_pos .= ijk_pos_s        
        
        x3c .= [ x1[ip], x2[jp], x3[kp] ]
        FirstTimeTraceGridOverOneLongRay(dirf, testRay_f, sg)         
        
        ijk_pos .= ijk_pos_s
        x3c .= [ x1[ip], x2[jp], x3[kp] ]
        FirstTimeTraceGridOverOneLongRay(dirb, testRay_b, sg)                                            
        
        # now, when the length of the ray is known, allocate arrOfLongRay[m,i,j,k]
        
        rayPairArr[m,ip,jp,kp] = RayPair(
            Array{Int32,1}(undef,3), 
            Array{Float64,1}(undef,3),

            LongRay1(testRay_f.numOfElm, Array{Int32}(undef,3,testRay_f.numOfElm), 
            Array{Float64}(undef,3,testRay_f.numOfElm)),
            
            LongRay1(testRay_b.numOfElm, Array{Int32}(undef,3,testRay_b.numOfElm), 
            Array{Float64}(undef,3,testRay_b.numOfElm))            
            )
                        
        rayPairArr[m,ip,jp,kp].ray_b.numOfElm = testRay_b.numOfElm  
        rayPairArr[m,ip,jp,kp].ray_f.numOfElm = testRay_f.numOfElm                      
        
        rayPairArr[m,ip,jp,kp].hed .= ijk_pos_s
        
        rayPairArr[m,ip,jp,kp].dir .= dirf

        for iter = 1:testRay_f.numOfElm       
            rayPairArr[m,ip,jp,kp].ray_f.ijkOfCellCrossed[:,iter] .= testRay_f.ijkOfCellCrossed[:,iter]
            rayPairArr[m,ip,jp,kp].ray_f.xyzPos[:,iter] .= testRay_f.xyzPos[:,iter]
        end
# @show testRay_b.numOfElm
        
        for iter = 1:testRay_b.numOfElm       
            rayPairArr[m,ip,jp,kp].ray_b.ijkOfCellCrossed[:,iter] .= testRay_b.ijkOfCellCrossed[:,iter]
            rayPairArr[m,ip,jp,kp].ray_b.xyzPos[:,iter] .= testRay_b.xyzPos[:,iter]
        end
        
        
    end
    
end


function TestRybHumAlongTheRay(rad)
    # pick a ray
    
    TraceGridAllocRayArr(rad, sg)
    
    ray =  rayPairArr[rand(1:rad.N_fre_ang), 
        ijk_pos_s[IX], ijk_pos_s[IY], ijk_pos_s[IZ]].ray_f
    
    den .= 2*10^3.
    opa .= 1.
    
    u1 = ProgonRybHumAlongTheRay(ray, sg, rad)
    u = ProgonRybHumAlongTheRayAppndxA(ray, sg, rad);    
#     @show u1 u
    Δ=LinearAlgebra.norm((u-u1)./u1)
    println( "relative difference between ProgonRybHumAlongTheRayAppndxA 
        and ProgonRybHumAlongTheRay:\n Δu/u=", Δ )    

    if Δ <= 1e-2
        println(" Test Passed ! ")    
    else
        error( " Test Failed !")
    end

    den.=opa.=0.;

end
